# Elixir: Processes
* In Elixir, all code runs inside __processes__. Processes are isolated from each other, run concurrent to one another and communicate via message passing. Processes are not only the basis for concurrency in Elixir, but they also provide the means for building distributed and fault-tolerant programs.

* Elixir’s processes should not be confused with operating system processes. Processes in Elixir are extremely lightweight in terms of memory and CPU (even compared to threads as used in many other programming languages). Because of this, __it is not uncommon to have tens or even hundreds of thousands of processes running simultaneously__(!!).

* In this chapter, we will learn about the basic constructs for spawning new processes, as well as sending and receiving messages between processes.

### Spawn
* Spawn is the basic mechanism for creating new processes. It takes a function which will execute in another process. 
* Notice that it returns a PID (process identifier). At this point the process you spawned in likely dead. The spawned process will execute the given function & exit after the function is done.

In [1]:
pid = spawn fn -> 1 + 2 end

#PID<0.221.0>

In [2]:
Process.alive?(pid)

false

In [3]:
# We can retrieve the PID of the current process by calling self/0:
self()

#PID<0.212.0>

In [4]:
Process.alive?(self())

true

### Send and Receive
* When a message is sent to a process, it is stored in a __process mailbox__. 
* __Receive__ searches the current process mailbox for a message that matches any of the given patterns. It supports guards and many clauses, such as case.

In [5]:
send self(), {:hello, "world"}

{:hello, "world"}

* If there is no message in the mailbox matching any of the patterns, the current process will wait until a matching message arrives. A timeout can also be specified.

In [6]:
receive do
  {:hello, msg} -> msg
after
  1000 -> 'nothing after 1 second'
  end

'nothing after 1 second'

* Let’s put it all together and send messages between processes.

In [7]:
parent = self()

spawn fn -> send(parent, {:hello, self()}) end

receive do
  {:hello, pid} -> "Got hello from #{inspect pid}"
  end

"Got hello from #PID<0.236.0>"

* __inspect/1__ converts a data structure’s internal representation into a string, typically for printing. When the receive block gets executed the sender process we have spawned may already be dead, as its only instruction was to send a message.

* While in the shell, you may find __flush/0__ quite useful. It flushes and prints all the messages in the mailbox.



In [8]:
send self(), :hello

:hello

In [9]:
flush()

:ok

* When executed in iex (command line):
![send-self-hello](pix/send-self-hello.png)

### Links
* Usually processes in Elixir are spawned as linked processes. If we want the failure in one process to propagate to another one, we should link them. This can be done with __spawn_link__. Let's see what happens when a process started with spawn fails:

![spawn-oops](pix/spawn-oops.png)

* It logged an error, but the parent process is still running. That's because processes are isolated. If we want a failure in one process to propagate to another, we need to link them with __spawn_link__.

![spawn-link-oops](pix/spawn-link-oop.png)

* Now we see a message saying the parent (shell) process received an EXIT signal from another process causing the shell to terminate. IEX detects this situation and starts a new shell session.

### [Processes](https://hexdocs.pm/elixir/Process.html)

* Elixir __processes__ are isolated and don’t share anything by default. Therefore, a failure in a process will never crash or corrupt the state of another process.

* __Links__, however, allow processes to establish a relationship in case of failure. We often link processes to supervisors which will detect when a process dies and start a new process in its place.

* While other languages require us to catch/handle exceptions, Elixir is fine with letting processes fail because we expect supervisors to properly restart our systems.

* __spawn__ and __spawn_link__ are the primitives for creating processes in Elixir. Usually we will use abstractions that build on top of them. Let’s see the most common one, called __tasks__.

### Tasks
* Tasks build on top of the spawn functions to provide better error reports and introspection.

![output](pix/tasks.png)

* Instead of spawn and spawn_link, we use __Task.start__ and __Task.start_link__ which return {:ok, pid} rather than just the PID. This is what enables tasks to be used in supervision trees. 

* Task provides convenience functions, like __Task.async__ and __Task.await__, and functionality to ease distribution.

### State
* If you are building an application that requires state, for example, to keep your application configuration, or you need to parse a file and keep it in memory, where do you store it?
* Processes are the most common answer. We can write processes that loop infinitely, maintain state, and send and receive messages. 
* As an example, let’s write a module that starts new processes that work as a key-value store in a file named kv.exs:

```
defmodule KV do
  def start_link do
    Task.start_link(fn -> loop(%{}) end)
  end

  defp loop(map) do
    receive do
      {:get, key, caller} ->
        send caller, Map.get(map, key)
        loop(map)
      {:put, key, value} ->
        loop(Map.put(map, key, value))
    end
  end
end```

* start_link starts a new process that runs loop/1, starting with an empty map. 
* loop/1 waits for messages and performs the appropriate action for each message. In the case of a :get message, it sends a message back to the caller and calls loop/1 again, to wait for a new message. While the :put message actually invokes loop/1 with a new version of the map, with the given key and value stored.

![kv-exs](pix/kv-exs.png)

* At first, the process map has no keys, so sending a :get message and then flushing the current process inbox returns nil. Let’s send a :put message and try it again:

![kv2-exs](pix/kv2-exs.png)

* Note how we can access & update this process's state by sending messages. An process that knows the PID can do the same.

* We also can register the PID, giving it a name, and allowing everyone that knows the name to send it messages.

![kv3-exs](pix/kv3-exs.png)

* Using processes to maintain state & name registration are common use cases in Elixir. Usually we will use an abstraction to create these patterns. For example, Elixir provides __[agents](https://hexdocs.pm/elixir/Agent.html)__:

![agents](pix/agents.png)